In [1]:
import pandas as pd
import numpy as np
import datetime

In [15]:
def EMA(df, base, target, period, alpha=False):
    """
    Function to compute Exponential Moving Average (EMA)
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        base : String indicating the column name from which the EMA needs to be computed from
        target : String indicates the column name to which the computed data needs to be stored
        period : Integer indicates the period of computation in terms of number of candles
        alpha : Boolean if True indicates to use the formula for computing EMA using alpha (default is False)
        
    Returns :
        df : Pandas DataFrame with new column added with name 'target'
    """

    con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])
    
    if (alpha == True):
        # (1 - alpha) * previous_val + alpha * current_val where alpha = 1 / period
        df[target] = con.ewm(alpha=1 / period, adjust=False).mean()
    else:
        # ((current_val - previous_val) * coeff) + previous_val where coeff = 2 / (period + 1)
        df[target] = con.ewm(span=period, adjust=False).mean()
    
    df[target].fillna(0, inplace=True)
    return df
def ATR(df, period, ohlc=['Open', 'High', 'Low', 'Close']):
    """
    Function to compute Average True Range (ATR)
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        period : Integer indicates the period of computation in terms of number of candles
        ohlc: List defining OHLC Column names (default ['Open', 'High', 'Low', 'Close'])
        
    Returns :
        df : Pandas DataFrame with new columns added for 
            True Range (TR)
            ATR (ATR_$period)
    """
    atr = 'ATR_' + str(period)

    # Compute true range only if it is not computed and stored earlier in the df
    if not 'TR' in df.columns:
        df['h-l'] = df[ohlc[1]] - df[ohlc[2]]
        df['h-yc'] = abs(df[ohlc[1]] - df[ohlc[3]].shift())
        df['l-yc'] = abs(df[ohlc[2]] - df[ohlc[3]].shift())
         
        df['TR'] = df[['h-l', 'h-yc', 'l-yc']].max(axis=1)
         
        df.drop(['h-l', 'h-yc', 'l-yc'], inplace=True, axis=1)

    # Compute EMA of true range using ATR formula after ignoring first row
    EMA(df, 'TR', atr, period, alpha=True)
    
    return df

def SuperTrend(df, period, multiplier, ohlc=['Open', 'High', 'Low', 'Close']):
    """
    Function to compute SuperTrend
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        period : Integer indicates the period of computation in terms of number of candles
        multiplier : Integer indicates value to multiply the ATR
        ohlc: List defining OHLC Column names (default ['Open', 'High', 'Low', 'Close'])
        
    Returns :
        df : Pandas DataFrame with new columns added for 
            True Range (TR), ATR (ATR_$period)
            SuperTrend (ST_$period_$multiplier)
            SuperTrend Direction (STX_$period_$multiplier)
    """

    ATR(df, period, ohlc=ohlc)
    atr = 'ATR_' + str(period)
    st = 'ST_' + str(period) + '_' + str(multiplier)
    stx = 'STX_' + str(period) + '_' + str(multiplier)
    
    """
    SuperTrend Algorithm :
    
        BASIC UPPERBAND = (HIGH + LOW) / 2 + Multiplier * ATR
        BASIC LOWERBAND = (HIGH + LOW) / 2 - Multiplier * ATR
        
        FINAL UPPERBAND = IF( (Current BASICUPPERBAND < Previous FINAL UPPERBAND) or (Previous Close > Previous FINAL UPPERBAND))
                            THEN (Current BASIC UPPERBAND) ELSE Previous FINALUPPERBAND)
        FINAL LOWERBAND = IF( (Current BASIC LOWERBAND > Previous FINAL LOWERBAND) or (Previous Close < Previous FINAL LOWERBAND)) 
                            THEN (Current BASIC LOWERBAND) ELSE Previous FINAL LOWERBAND)
        
        SUPERTREND = IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close <= Current FINAL UPPERBAND)) THEN
                        Current FINAL UPPERBAND
                    ELSE
                        IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close > Current FINAL UPPERBAND)) THEN
                            Current FINAL LOWERBAND
                        ELSE
                            IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close >= Current FINAL LOWERBAND)) THEN
                                Current FINAL LOWERBAND
                            ELSE
                                IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close < Current FINAL LOWERBAND)) THEN
                                    Current FINAL UPPERBAND
    """
    
    # Compute basic upper and lower bands
    df['basic_ub'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 + multiplier * df[atr]
    df['basic_lb'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 - multiplier * df[atr]

    # Compute final upper and lower bands
    df['final_ub'] = 0.00
    df['final_lb'] = 0.00
    for i in range(period, len(df)):
        df['final_ub'].iat[i] = df['basic_ub'].iat[i] if df['basic_ub'].iat[i] < df['final_ub'].iat[i - 1] or df[ohlc[3]].iat[i - 1] > df['final_ub'].iat[i - 1] else df['final_ub'].iat[i - 1]
        df['final_lb'].iat[i] = df['basic_lb'].iat[i] if df['basic_lb'].iat[i] > df['final_lb'].iat[i - 1] or df[ohlc[3]].iat[i - 1] < df['final_lb'].iat[i - 1] else df['final_lb'].iat[i - 1]
       
    # Set the Supertrend value
    df[st] = 0.00
    for i in range(period, len(df)):
        df[st].iat[i] = df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[i] <= df['final_ub'].iat[i] else \
                        df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[i] >  df['final_ub'].iat[i] else \
                        df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] >= df['final_lb'].iat[i] else \
                        df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] <  df['final_lb'].iat[i] else 0.00 
                 
    # Mark the trend direction up/down
    df[stx] = np.where((df[st] > 0.00), np.where((df[ohlc[3]] < df[st]), 'down',  'up'), np.NaN)

    # Remove basic and final bands from the columns
    df.drop(['basic_ub', 'basic_lb', 'final_ub', 'final_lb'], inplace=True, axis=1)
    
    df.fillna(0, inplace=True)

    return df

In [47]:
df = pd.read_csv("NIFTY BANK 5min.csv",parse_dates=True)

In [48]:
df = SuperTrend(df, 9, 2, ohlc=['Open', 'High', 'Low', 'Close'])


In [62]:
starttime = '14:30'
endtime = '15:00'
Dates = df.Date.tolist()
Super = df.ST_9_2.tolist()
Price = df.Close.tolist()
Signal = df.STX_9_2.tolist()
Time = []
temp = []
for i in range(len(Dates)):
    temp = Dates[i][9:]
    Time.append(temp)
start = Time.index(starttime)
end = Time.index(endtime)

In [63]:
onlydates = []
temp = []
for i in range(len(Dates)):
    temp = Dates[i][0:9]
    onlydates.append(temp)



In [64]:
Type =   []
Entry =  []
DATES =  []
collect= []
TimeRange = Time[start:end +1]
for i in range(len(Time)):
    if Time[i] in TimeRange:
        if Signal[i] != Signal[i-1]:
            Entry.append(Price[i])
            Type.append(Signal[i])
            DATES.append(Dates[i])
            collect.append(i)


            

In [65]:
Exit=[]
for i in collect:
    x = i
    for j in range(x,x+150):
        if onlydates [j] != onlydates[j-1]:
            Exit.append(Price[j])
            break
        if Signal[j] != Signal[j-1]:
            Exit.append(Price[j+1])
            break            

In [66]:
PnL =[]
for i in range(len(Type)):
    if Type[i] == 'up':
        value = Exit[i] - Entry[i]
        PnL.append(value)
    else: 
        value = Entry[i] - Exit[i]
        PnL.append(value)
for i in range(len(Type)):
    if Type[i] == 'down':
        Type[i] ='Sell'
    else:
        Type[i] = 'Buy'
Supertrend = pd.DataFrame({
     'Date' : DATES,
     'Entry': Entry,     
     'Type': Type,
     'Exit': Exit,
     'PnL': PnL
    })          
Supertrend.PnL.sum()   

7.650000000001455

In [67]:
Time.index('15:30')

6075

In [68]:
x = df.Date[1]

In [69]:
x

'3/5/2019 9:20'

In [70]:
x[9:]

'9:20'

In [72]:
Time

['9:15',
 '9:20',
 '9:25',
 '9:30',
 '9:35',
 '9:40',
 '9:45',
 '9:50',
 '9:55',
 '10:00',
 '10:05',
 '10:10',
 '10:15',
 '10:20',
 '10:25',
 '10:30',
 '10:35',
 '10:40',
 '10:45',
 '10:50',
 '10:55',
 '11:00',
 '11:05',
 '11:10',
 '11:15',
 '11:20',
 '11:25',
 '11:30',
 '11:35',
 '11:40',
 '11:45',
 '11:50',
 '11:55',
 '12:00',
 '12:05',
 '12:10',
 '12:15',
 '12:20',
 '12:25',
 '12:30',
 '12:35',
 '12:40',
 '12:45',
 '12:50',
 '12:55',
 '13:00',
 '13:05',
 '13:10',
 '13:15',
 '13:20',
 '13:25',
 '13:30',
 '13:35',
 '13:40',
 '13:45',
 '13:50',
 '13:55',
 '14:00',
 '14:05',
 '14:10',
 '14:15',
 '14:20',
 '14:25',
 '14:30',
 '14:35',
 '14:40',
 '14:45',
 '14:50',
 '14:55',
 '15:00',
 '15:05',
 '15:10',
 '15:15',
 '15:20',
 '15:25',
 '9:15',
 '9:20',
 '9:25',
 '9:30',
 '9:35',
 '9:40',
 '9:45',
 '9:50',
 '9:55',
 '10:00',
 '10:05',
 '10:10',
 '10:15',
 '10:20',
 '10:25',
 '10:30',
 '10:35',
 '10:40',
 '10:45',
 '10:50',
 '10:55',
 '11:00',
 '11:05',
 '11:10',
 '11:15',
 '11:20',
 '11:25'